# Team Project 1 - MATH 420
## James Boggs and Camilo Velez R
### NYC Covid Data

In [2]:
# activate env and instantiate pkgs
using Pkg
Pkg.activate("p1")
Pkg.instantiate()

  Activating new project at `~/m420p1/src/p1`


In [4]:
using CSV, DataFrames, Dates # import necessary pkgs

In [5]:
df = CSV.read("../resources/data.csv", DataFrame) # entire df

v_infected = values(df[1, 13:end]) # vector of infected numbers
infected_dates = names(df[1, 13:end]) # vector of infected_dates
@assert length(v_infected) == length(infected_dates)

46

In [6]:
findfirst([x>=5 for x in v_infected])

46

$T_{0} = 46$